In [1]:
import sys
# sys.path.append("D:\thesis\thesis\rllib")

import rllib as rl
import json
import joblib
import numpy as np
import torch
dataset_train = "./data_preprossing/output/train_cleaned.csv"
dataset_test = "./data_preprossing/output/test_cleaned.csv"
symptom_nlice_file ="./data/basic/symptom_nlice.json"
# symptom_map_file = "./data/basic/symptoms_db.json"
symptom_map_file = "./data_preprossing/output/basic/symptom_db_v1.json"
condition_map_file = "./data/basic/conditions_db.json"
body_parts="./data_preprossing/output/basic/body-parts-enc.json"
excitation_enc="./data_preprossing/output/basic/excitation_encoding.json"
frequency_enc="./data_preprossing/output/basic/frequency_encoding.json"
nature_enc="./data_preprossing/output/basic/nature_encoding.json"
vas_enc="./data_preprossing/output/basic/vas_encoding.json"
onset_enc="./data_preprossing/output/basic/onset_encoding.json"
duration_enc="./data_preprossing/output/basic/duration_encoding.json"
# clf_file = "data/basic/data/output/rf/rf_clf.joblib"
# clf_file = "data/basic/data/output/rf/rf_clf.joblib"
clf_file = "./symtom_models/output/nb_nlice/nb_serialized_sparse.joblib"
clf_data = joblib.load(clf_file)
clf = clf_data.get('clf')
import datetime
import sys
from pathlib import Path
curr_path = str(Path().absolute())
parent_path = str(Path().absolute().parent)
sys.path.append(parent_path) # add current terminal path to sys.path
curr_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # obtain current time
test = "./data_preprossing/output/data/total_1k_min3.csv"
test_v1 = "./data_preprossing/output/data/total_50.csv"
# with open(symptom_map_file) as fp:
#             symptoms = json.load(fp)
#             sorted_symptoms = sorted(symptoms.keys())
#             symptom_map = {code: idx for idx, code in enumerate(sorted_symptoms)}
# with open(symptom_map_file, "w") as fp:
#     json.dump(symptom_map, fp, indent=4)


In [2]:
env = rl.environment.RLBasicMedEnvContext(
    data_file=test,
    symptom_map_file=symptom_map_file,
    condition_map_file=condition_map_file,
    body_parts=body_parts,
    excitation_enc=excitation_enc,
    frequency_enc=frequency_enc,
    nature_enc=nature_enc, 
    vas_enc=vas_enc, 
    onset_enc=onset_enc, 
    duration_enc=duration_enc,
    clf=clf,
    classifer="nb",
    max_turn=20,
    epoch =16000
)

In [3]:
input_dim = env.num_symptoms*8
output_dim = env.num_symptoms + 33
print(env.num_symptoms)
layer_config = [
    [input_dim, 128],
    [128, 64],
    [64, 48],
    [48, 32],
    [32, 32],
    [32, 48],
    [48, 64],
    [64, output_dim]
]

learning_start = 1280
batch_size = 64
target_update = 1280
replay_capacity = 100000

61


In [8]:
agent = rl.agent.MedAgent(
    env,
    layer_config=layer_config,
    learning_start=learning_start,
    batch_size=batch_size,
    target_update=target_update,
    replay_capacity=replay_capacity,
    debug=False,
    train = True, 
    input_dim = input_dim,
    output_dim = output_dim,
    context = False
)

In [9]:
bench = rl.bench.MedBench(agent,33000)
bench.run_trial(debug=False)
cfg = rl.config.DQNConfig()
rl.common.make_dir(cfg.result_path,cfg.model_path)
agent.save(cfg.model_path)


In [10]:

cfg = rl.config.DQNConfig()
rewards,success,average_rewards,average_steps,loss =bench.rewardList,bench.get_success_rate(),bench.get_average_rewards(),bench.get_average_turn(),bench.get_loss()
print(success)
print(average_rewards,average_steps)
env1 = rl.environment.RLBasicMedEnvContext(
    data_file=test,
    symptom_map_file=symptom_map_file,
    condition_map_file=condition_map_file,
    symptom_nlice_file=symptom_nlice_file,
    body_parts=body_parts,
    excitation_enc=excitation_enc,
    frequency_enc=frequency_enc,
    nature_enc=nature_enc, 
    vas_enc=vas_enc, 
    onset_enc=onset_enc, 
    duration_enc=duration_enc,
    clf=clf,
    classifer="nb",
    max_turn=20,
    epoch = 16000
)
# agent.policy_network.load_state_dict(torch.load('./outputs/DQN/20211125-160309/models/dqn_checkpoint_policy.pth'))
agent.env = env1
agent.train = False
agent.eps_start = 0
agent.eps_end = 0
agent.debug = False
testbench = rl.bench.MedBenchEval(agent,16000)
testbench.run_trial(debug=False)
rewards,success,average_rewards,average_steps,hint_rate =testbench.rewardList,testbench.get_success_rate(),testbench.get_average_rewards(),testbench.get_average_turn(),testbench.get_hint_rate()
print(testbench.success)
print(success)
print(average_rewards,average_steps)
print(hint_rate)

0.5380303030303031
36.97690909090909 3.036
11262
0.703875
41.771 2.1716875
0.5674375


In [ ]:
agent_test = rl.agent.MedAgent(
    env1,
    layer_config=layer_config,
    learning_start=learning_start,
    batch_size=batch_size,
    target_update=target_update,
    replay_capacity=replay_capacity,
    debug=False,
    train = False
)
# torch.save(agent.policy_network.state_dict(), cfg.model_path+'/dqn_checkpoint_policy.pth')
# agent_test.policy_network.load_state_dict(agent.target_network.state_dict())
# agent_test.policy_network.load_state_dict(agent.policy_network.state_dict())
# for target_param, param in zip(agent_test.policy_network.parameters(),agent.policy_network.parameters()): # 复制参数到目标网路targe_net
#             target_param.data.copy_(param.data)
# for target_param, param in zip(agent_test.policy_network.parameters(),agent.target_network.parameters()): # 复制参数到目标网路targe_net
#             target_param.data.copy_(param.data)
agent_test.eps_start = 0
agent_test.eps_end = 0
# print(cfg.model_path)
# agent_test.load(cfg.model_path)
agent_test.policy_network.load_state_dict(torch.load('./outputs/DQN/20211123-114930/models/dqn_checkpoint_policy.pth'))
testbench = rl.bench.MedBenchEval(agent_test,1000)
testbench.run_trial(debug=False)
rewards,success,average_rewards,average_steps =testbench.rewardList,testbench.get_success_rate(),testbench.get_average_rewards(),testbench.get_average_turn()
print(success)
print(average_rewards,average_steps)


In [19]:
test = "./data_preprossing/output/datas/min3/Asthma.csv"
env1 = rl.environment.RLBasicMedEnvNOPATIENT(
    data_file=test,
    symptom_map_file=symptom_map_file,
    condition_map_file=condition_map_file,
    symptom_nlice_file=symptom_nlice_file,
    body_parts=body_parts,
    excitation_enc=excitation_enc,
    frequency_enc=frequency_enc,
    nature_enc=nature_enc, 
    vas_enc=vas_enc, 
    onset_enc=onset_enc, 
    duration_enc=duration_enc,
    clf=clf,
    classifer="nb",
    max_turn=20,
    epoch = 1000
)


In [20]:
agent.env = env1
agent.train = False
agent.eps_start = 0
agent.eps_end = 0
agent.debug = False
testbench = rl.bench.MedBenchEval(agent,1000)
testbench.run_trial(debug=False)
rewards,success,average_rewards,average_steps,hint_rate =testbench.rewardList,testbench.get_success_rate(),testbench.get_average_rewards(),testbench.get_average_turn(),testbench.get_hint_rate()
print(success)
print(average_rewards,average_steps)
print(hint_rate)

0.0
-18.0 21.0
0.0


In [19]:
def find_symptom(index):
        symptoms=[]
        for s in index:
            if(s%8==0):
                symptoms.append(s/8)
        return symptoms


agent.env = env1
agent.train = False
agent.eps_start = 0
agent.eps_end = 0
agent.debug = False
testbench = rl.bench.MedBenchEval(agent,1)
testbench.run_trial(debug=False)
print(agent.env.patient)
print(agent.env.patient.condition)
print(np.where(agent.env.patient.symptoms==1)[0])
print(find_symptom(np.where(agent.env.patient.symptoms==1)[0]))
print(agent.env.inquiry_list)
rewards,success,average_rewards,average_steps =testbench.rewardList,testbench.get_success_rate(),testbench.get_average_rewards(),testbench.get_average_turn()
print(success)
print(average_rewards,average_steps)

AiMedPatient(symptoms=array([ 0,  0,  0,  0,  0,  0,  0,  0,  1, 11,  1,  1,  1,  1,  1,  1,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  0,  0,  0,  0,  0,
   

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from matplotlib.font_manager import FontProperties

def plot_loss(loss,tag="train",env="milestone1",algo = "DQN"):
    sns.set()
    plt.title("average learning curve of {} for {}".format(algo,env))
    plt.xlabel('epsiodes')
    plt.plot(loss,label='loss')
    plt.legend()


def plot_rewards(rewards,tag="train",env="milestone1",algo = "DQN"):
    sns.set()
    plt.title("average learning curve of {} for {}".format(algo,env))
    plt.xlabel('epsiodes')
    plt.plot(rewards,label='reward')
    plt.legend()


In [ ]:
# plot_rewards(rewards)
eval = rewards
success = 0
plot_rewards(eval)
